In [264]:
import pandas as pd
import numpy as np
import re

##### Check scraped 700 pages

In [265]:
df = pd.read_csv('/Users/judy/Desktop/WebScp/GitWebScp/kicktraq/kicktraq/kicktraq.csv')

In [266]:
df.shape[0]/15 #15 projects per page


699.6666666666666

In [267]:
df.shape[0]

10495

###### Rescraped to include try except=None on backers

In [153]:
df2 = pd.read_csv('/Users/judy/Desktop/WebScp/GitWebScp/kicktraq/kicktraq/kicktraq2.csv')

In [139]:
df2.shape[0]/15 # check if 700 pages

699.6666666666666

##### Check to see if any backers null

In [140]:
df2.loc[df2['backers'].isnull()] #no nulls

,backers,category,currency,description,goal,main_category,names,pledged,proj_details,status


##### Check to see if two scrapes returned same number rows

In [141]:
df2.shape[0] == df.shape[0]

False

# Cleaning and renaming colunms

##### Using df2 reassign as dfn for newly scraped kicktraq - to distinguish between older kaggle dataset (dfo)

In [268]:
dfn = pd.read_csv('/Users/judy/Desktop/WebScp/GitWebScp/kicktraq/kicktraq/kicktraq2.csv')

In [269]:
dfn.sample(5)

,backers,category,currency,description,goal,main_category,names,pledged,proj_details,status
908,51,games|playing cards,$,A mystical and strange tarot deck full of uniq...,9000.0,games,Mystic Garbage Tarot,3532.0,"\n\t\t\t\t\tBackers: 51,\n\t\t\t\t\tFunding: $...",Closed
7486,93,publishing|children's books,£,A beautifully illustrated rhyming children's p...,1000.0,publishing,Our Dave: A Rhyming Children's Picture Book,2206.0,"\n\t\t\t\t\tBackers: 93,\n\t\t\t\t\tFunding: £...",Closed
6003,2,NaN,€,Unique custom made tshirts for all the rollers...,300.0,design,I´M A ROLLER. Unique tshirts for roller bladin...,11.0,"\n\t\t\t\t\tBackers: 2,\n\t\t\t\t\tFunding: €1...",Closed
9929,217,comics|anthologies,$,The second volume in the anthology horror anth...,5000.0,comics,Tales From The Cryptic Closet 2,7788.0,"\n\t\t\t\t\tBackers: 217,\n\t\t\t\t\tFunding: ...",Closed
2421,9,music|rock,NaN,Re-printing the Catherines Cathedral CDs on vi...,NaN,music,"Catherines Cathedral ""Flowerdust"" vinyl!",NaN,"\n\t\t\t\t\tBackers: 9,\n\t\t\t\t\tFunding: kr...",Closed


##### Extracting date portion of proj_details into 'campaign_dates' col - there are no missing values 

In [270]:
### KEEP THIS! THIS GETS campaign dates as a chunck will play with to get datetimes
dfn['campaign_dates'] = [re.findall('Campaign (Dates: .+)', el)[0] for el in dfn['proj_details']]    

##### 'launched' col from campaign_dates

In [271]:
# creates list of tuples with Month, day , year
start_date = [re.findall('Dates: (\D*) (\d*).+\((\d+)\)', el)[0] for el in dfn['campaign_dates']]

In [272]:
# join tuple elements into string, now list of strings
start_date = [' '.join(date) for date in start_date]

In [273]:
# create column 'launched' with datetime object from list
dfn['launched']= [pd.datetime.strptime(s, '%B %d %Y') for s in start_date]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


In [274]:
dfn.head(5)

,backers,category,currency,description,goal,main_category,names,pledged,proj_details,status,campaign_dates,launched
0,10,food|drinks,$,A do it yourself boba tea kit that allows you ...,3000.0,food,Boba Era: Boba From Home,401.0,"\n\t\t\t\t\tBackers: 10,\n\t\t\t\t\tFunding: $...",Cancelled,"Dates: June 3rd -> August 2nd (2020),",2020-06-03
1,1,fashion|apparel,$,Sophisticated Mexican old school style apparel.,16578.0,fashion,Mr Cochino Apparel,1.0,"\n\t\t\t\t\tBackers: 1,\n\t\t\t\t\tFunding: $1...",Cancelled,"Dates: June 26th -> August 2nd (2020),",2020-06-26
2,1,fashion|apparel,NaN,RotenStein Clothing Co. Una nueva marca de rop...,NaN,fashion,RotenStein Clothing Co. (Arte en tu ropa),NaN,"\n\t\t\t\t\tBackers: 1,\n\t\t\t\t\tFunding: MX...",Cancelled,"Dates: June 3rd -> August 2nd (2020),",2020-06-03
3,1,games|video games,$,"Z-City, A Zombie based Survival-Battle Royale ...",5000.0,games,Z-City Survival-Battle Royale,1.0,"\n\t\t\t\t\tBackers: 1,\n\t\t\t\t\tFunding: $1...",Cancelled,"Dates: July 3rd -> August 2nd (2020),",2020-07-03
4,0,comics|graphic novels,$,A Samurai Horror Graphic Novel,7500.0,comics,Death Kanji,0.0,"\n\t\t\t\t\tBackers: 0,\n\t\t\t\t\tFunding: $0...",Cancelled,"Dates: July 3rd -> August 2nd (2020),",2020-07-03


##### 'deadline' same as 'launched' above

In [275]:
end_date = [re.findall('-> (\D+) (\d+).+\((\d+)\)', el)[0] for el in dfn['campaign_dates']]

In [276]:
end_date = [' '.join(date) for date in end_date]

In [277]:
# drop campaign dates
dfn['deadline']= [pd.datetime.strptime(s, '%B %d %Y') for s in end_date]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


In [253]:
dfn.sample(5)

,backers,category,currency,description,goal,main_category,names,pledged,proj_details,status,campaign_dates,launched,deadline
1103,57,design|product design,$,The Recon Series Precision Timepiece,2000.0,design,Analog Field Watch at an Unbelievable Price!,3051.0,"\n\t\t\t\t\tBackers: 57,\n\t\t\t\t\tFunding: $...",Closed,"Dates: March 19th -> April 12th (2020),",2020-03-19,2020-04-12
4136,184,games|video games,€,3D point-and-click puzzle adventure game,10000.0,games,IllusiveGate,5778.0,"\n\t\t\t\t\tBackers: 184,\n\t\t\t\t\tFunding: ...",Closed,"Dates: April 19th -> May 19th (2020),",2020-04-19,2020-05-19
8605,122,games|tabletop games,$,"A series of RPG maps of a huge Galleon, an Isl...",2149.0,games,"Collection of Rpg Maps, a huge galleon, and 5t...",5592.0,"\n\t\t\t\t\tBackers: 122,\n\t\t\t\t\tFunding: ...",Closed,"Dates: June 9th -> July 5th (2020),",2020-06-09,2020-07-05
3286,4,film & video|experimental,$,Film festival funding for an experimental arth...,600.0,film & video,Film Festival Funding: Ream III,71.0,"\n\t\t\t\t\tBackers: 4,\n\t\t\t\t\tFunding: $7...",Closed,"Dates: March 9th -> May 8th (2020),",2020-03-09,2020-05-08
2004,7,food|farms,$,"Building a small, all natural, organic farm pr...",10000.0,food,Scholz's Farm!,1196.0,"\n\t\t\t\t\tBackers: 7,\n\t\t\t\t\tFunding: $1...",Closed,"Dates: March 24th -> April 23rd (2020),",2020-03-24,2020-04-23


##### Extracting funding portion of proj_details into 'funding' col also deals with missing values

In [278]:
##### KEEP THIS HOW WE GOT THE FUNDING STRING EVEN IF MISSING
dfn['funding'] = [None if re.findall('Funding: (.*)', el)==[] else re.findall('Funding: (.*)', el)[0] for el in dfn['proj_details']] 

In [279]:
dfn['funding']

0           $401 of $3,000 (,13% funded,),
1             $1 of $16,578 (,0% funded,),
2        MX$19 of MX$73,000 (,0% funded,),
3              $1 of $5,000 (,0% funded,),
4              $0 of $7,500 (,0% funded,),
                       ...                
10490         $1 of $12,100 (,0% funded,),
10491         €1 of €50,000 (,0% funded,),
10492              $0 of $1 (,0% funded,),
10493         $17 of $5,000 (,0% funded,),
10494     $1,301 of $2,090 (,62% funded,),
Name: funding, Length: 10495, dtype: object

In [280]:
dfn.sample(5)

,backers,category,currency,description,goal,main_category,names,pledged,proj_details,status,campaign_dates,launched,deadline,funding
8066,48,design|product design,$,"Organize your Essentials - keys, cards, cash ...",1400.0,design,The Artisanal Credit Card sized EDC Wallet 2.0...,3099.0,"\n\t\t\t\t\tBackers: 48,\n\t\t\t\t\tFunding: $...",Closed,"Dates: June 1st -> July 1st (2020),",2020-06-01,2020-07-01,"$3,099 of $1,400 (,221% funded,),"
5342,235,fashion|apparel,$,Snow Rabbit Co is creating minimalistic clothi...,3200.0,fashion,Celestial Focus by Snow Rabbit Co.,18270.0,"\n\t\t\t\t\tBackers: 235,\n\t\t\t\t\tFunding: ...",Closed,"Dates: April 17th -> June 1st (2020),",2020-04-17,2020-06-01,"$18,270 of $3,200 (,570% funded,),"
3239,59,photography|fine art,€,Photography live streams from the darkroom,1200.0,photography,"Darkstreams, photography livestreams",1727.0,"\n\t\t\t\t\tBackers: 59,\n\t\t\t\t\tFunding: €...",Closed,"Dates: April 7th -> May 7th (2020),",2020-04-07,2020-05-07,"€1,727 of €1,200 (,143% funded,),"
1380,8,technology|apps,€,Application mobile permettant de commander des...,8000.0,technology,MoovTime Payment,1231.0,"\n\t\t\t\t\tBackers: 8,\n\t\t\t\t\tFunding: €1...",Closed,"Dates: March 2nd -> April 16th (2020),",2020-03-02,2020-04-16,"€1,231 of €8,000 (,15% funded,),"
5390,2,fashion|apparel,$,"We a are a branch of ""Velo Squad"" trying to cr...",2000.0,fashion,ATYPIKAL,51.0,"\n\t\t\t\t\tBackers: 2,\n\t\t\t\t\tFunding: $5...",Closed,"Dates: April 2nd -> June 1st (2020),",2020-04-02,2020-06-01,"$51 of $2,000 (,2% funded,),"


### Splitting category

In [148]:
#### KEEP
dfn_cat = dfn['category'].str.split('|',expand=True) #this one and merge to data frame

###### concat dfn and dfn_cat, drop 0 (name of the first component of split), rename second new column 'sub_category'

In [149]:
#### KEEP
dfn = pd.concat([dfn, dfn_cat], axis=1).drop(0,1).rename(columns = {1:'sub_category'})

In [150]:


dfn.sample(5)

,backers,category,currency,description,goal,main_category,names,pledged,proj_details,status,campaign_dates,funding,sub_category
10132,1095,fashion|apparel,$,"Quick-dry, antibacterial, and eco-friendly und...",10000.0,fashion,"SOMI: Undies that Lounge, Gym, & Swim",105659.0,"\n\t\t\t\t\tBackers: 1095,\n\t\t\t\t\tFunding:...",Closed,"June 17th -> July 21st (2020),","$105,659 of $10,000 (,1056% funded,),",apparel
2354,26,fashion|accessories,€,35mm wide rainbow metal pins with holographic ...,350.0,fashion,Certified Crazy Cat Lover and Certified Crazy ...,396.0,"\n\t\t\t\t\tBackers: 26,\n\t\t\t\t\tFunding: €...",Closed,"March 27th -> April 26th (2020),","€396 of €350 (,113% funded,),",accessories
9100,345,comics|comic books,$,A pulse-pounding sci-fi mystery about a scient...,6000.0,comics,Duplicant 1-3: A Dystopian Biopunk Thriller,9546.0,"\n\t\t\t\t\tBackers: 345,\n\t\t\t\t\tFunding: ...",Closed,"June 10th -> July 10th (2020),","$9,546 of $6,000 (,159% funded,),",comic books
2813,12,art|video art,£,"A feature film about Mary I, the first queen r...",3000.0,art,Seed funding for feature film 'THE FIRST QUEEN',3000.0,"\n\t\t\t\t\tBackers: 12,\n\t\t\t\t\tFunding: £...",Closed,"March 10th -> May 1st (2020),","£3,000 of £3,000 (,100% funded,),",video art
2705,11,NaN,$,An all-play card game,10000.0,games,BULL TISH! The game that's absolutely FULL OF IT!,320.0,"\n\t\t\t\t\tBackers: 11,\n\t\t\t\t\tFunding: $...",Closed,"March 31st -> April 30th (2020),","$320 of $10,000 (,3% funded,),",NaN


In [103]:
# dfn['main_category'] = [el.split('|')[0] for el in dfn['category']] # would work if had no missing
# dfn['sub_category'] = [el.split('|')[1] for el in dfn['category']]

In [10]:
dfn.columns

Index(['backers', 'category', 'currency', 'description', 'goal',
       'main_category', 'names', 'pledged', 'proj_details', 'status'],
      dtype='object')

In [72]:
# test_list = ['\n\t\t\t\t\tBackers: 10,\n\t\t\t\t\tFunding: $401 of $3,000 (,13% funded,),\n\t\t\t\t\tCampaign Dates: June 3rd -> August 2nd (2020),\n\t\t\t\t\tClosed: Sunday August 2nd, 2020,\n\t\t\t\t',
#  '\n\t\t\t\t\tBackers: 1,\n\t\t\t\t\tCampaign Dates: June 26th -> August 2nd (2020),\n\t\t\t\t\tClosed: Sunday August 2nd, 2020,\n\t\t\t\t',
#  '\n\t\t\t\t\tBackers: 1,\n\t\t\t\t\tFunding: MX$19 of MX$73,000 (,0% funded,),\n\t\t\t\t\tCampaign Dates: June 3rd -> August 2nd (2020),\n\t\t\t\t\tClosed: Sunday August 2nd, 2020,\n\t\t\t\t',]

##### Changing currency

In [147]:
dfn['currency'].value_counts() # see what currency we have

$    6661
€    1560
£    1446
¥      71
Name: currency, dtype: int64

##### creating currency exchange data frame

In [148]:
symb_dict = {'currency':['$','€','£', '¥'], 'currency_code':['USD', 'EUR', 'GPB', 'JPY'], 'exchange_rt_to_USD': [1, 1.16, 1.28, 0.0094]}
df_cur_sym = pd.DataFrame(symb_dict)
df_cur_sym


,currency,currency_code,exchange_rt_to_USD
0,$,USD,1.0000
1,€,EUR,1.1600
2,£,GPB,1.2800
3,¥,JPY,0.0094


##### merge dfn with currency exchange

In [149]:
dfn = pd.merge(dfn, df_cur_sym, how='left', on='currency')

,backers,category,currency,description,goal,main_category,names,pledged,proj_details,status,currency_code,exchange_rt_to_USD
5394,52,fashion|apparel,$,It's time for an Austin icon to finally give y...,9000.0,fashion,Pre-order Lucy in Disguise with Diamonds T-shirts,1736.0,"\n\t\t\t\t\tBackers: 52,\n\t\t\t\t\tFunding: $...",Closed,USD,1.0
8952,54,games|tabletop games,$,An RPG splat book using the OGL for the greate...,12500.0,games,The Elegant Fount: More Creativity for your Dn...,2212.0,"\n\t\t\t\t\tBackers: 54,\n\t\t\t\t\tFunding: $...",Closed,USD,1.0
10116,16,NaN,NaN,Promoting cleanliness at the tip of your fingers,NaN,design,LuftGuard™ | Hands-Free Sprayer and Dispenser,NaN,"\n\t\t\t\t\tBackers: 16,\n\t\t\t\t\tFunding: H...",Closed,NaN,NaN
8078,59,comics|graphic novels,$,"""Chapter 1: Reflections""",2500.0,comics,"SOUL, The Graphic Novel",3540.0,"\n\t\t\t\t\tBackers: 59,\n\t\t\t\t\tFunding: $...",Closed,USD,1.0
7856,73,fashion|apparel,$,Wearable Billboards for Goodness Sake. Worn to...,5500.0,fashion,ShirtBOARDS,7597.0,"\n\t\t\t\t\tBackers: 73,\n\t\t\t\t\tFunding: $...",Closed,USD,1.0


In [154]:
dfn.sample(5)

,backers,category,currency,description,goal,main_category,names,pledged,proj_details,status,currency_code,exchange_rt_to_USD
2931,5,NaN,€,Louis Lagrange wants to inspire a lifestyle by...,500.0,design,Redefining Italian Luxury Classic Bracelet - L...,181.0,"\n\t\t\t\t\tBackers: 5,\n\t\t\t\t\tFunding: €1...",Cancelled,EUR,1.16
7379,128,games|playing cards,$,An innovative 80-card Oracle Deck integrating ...,2900.0,games,Soul Wisdom Oracle Cards,7206.0,"\n\t\t\t\t\tBackers: 128,\n\t\t\t\t\tFunding: ...",Closed,USD,1.00
7046,25,crafts|stationery,$,A set of Greek greeting cards and stationery.,1000.0,crafts,Illustrated Cards by Vassi M Haros,1097.0,"\n\t\t\t\t\tBackers: 25,\n\t\t\t\t\tFunding: $...",Closed,USD,1.00
9386,1,fashion|apparel,€,una marca de ropa basada en mi mascota (americ...,2500.0,fashion,Amerdican Blue,1.0,"\n\t\t\t\t\tBackers: 1,\n\t\t\t\t\tFunding: €1...",Closed,EUR,1.16
2361,4,fashion|footwear,NaN,"PetStep, tenis personalizados pintados a mano,...",NaN,fashion,"PetStep, tenis personalizados pintados a mano",NaN,"\n\t\t\t\t\tBackers: 4,\n\t\t\t\t\tFunding: MX...",Closed,NaN,NaN


#### updating status column to include fail and success, percent funded, usd pledged, usd goal

In [93]:
df2_sort_names=df2.sort_values('names')['names']
df2_sort_names=df2_sort_names.dropna(axis=0, how='any')
df2_sort_names.shape[0]

10493

In [92]:
merged_names = pd.merge(df_sort_names, df2_sort_names, how = 'outer', on='names')
merged_names.shape[0]/15

712.3333333333334

In [ ]:
merged_names.loc[merged_names]

In [73]:
here = ['! EMPOWER MY POTENTIAL - DEVOUR MY MENTAL !',
 '" " -> Quotidian - a poetry debut collection',]
there = ['! EMPOWER MY POTENTIAL - DEVOUR MY MENTAL !',
 '" " -> Quotidian - a poetry debut collection',]
here == there

True

In [87]:
df2_sort_names == df_sort_names

False

In [ ]:
# month_dict = {'January':'01', 'February':'02', 'March':'03', 'April':'04', 'May':'05', 'June':'06', 'July':'07', 'August':'08', 'September':'09', 'October':'10', 'November':'11', 'December':'12'}

In [17]:
# category = ['/categories/main/']
# main_category = ''
# if len(category) == 1:
#     main_category = category[0].strip('/').replace('categories/','')
#     category = None
# else:
#     main_category = category[0].strip('/').replace('categories/','')
#     category = category[1].strip('/').replace('categories/','').replace('/','|')
# print(main_category)
# print(category)

main
None


In [86]:
df = pd.read_csv('/Users/judy/Desktop/WebScp/GitWebScp/data/Web_Scraping_Proj/ks-projects-201801.csv')
df.sample(20)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
109037,1554153481,The SIFF Orchestra,Classical Music,Music,USD,2012-05-08,5000.0,2012-04-06 10:51:06,5737.00,successful,44,US,5737.00,5737.00,5000.00
188708,1960221447,This Year Funk,Dance,Dance,USD,2015-05-20,400.0,2015-05-05 00:28:56,0.00,failed,0,US,0.00,0.00,400.00
170418,1866985640,"The Tin, a Boston police drama",Shorts,Film & Video,USD,2011-11-16,8000.0,2011-10-12 17:34:36,8425.77,successful,53,US,8425.77,8425.77,8000.00
327837,739749652,Omnia Playing Cards,Playing Cards,Games,USD,2015-07-02,10000.0,2015-06-02 12:52:45,44147.00,successful,784,US,44147.00,44147.00,10000.00
102457,1520608340,Walkin' After Midnight.,Sculpture,Art,USD,2012-01-17,1000.0,2011-12-20 10:40:01,1000.01,successful,19,US,1000.01,1000.01,1000.00
374915,980646100,Uniquely Me - A Children's Book For Kids With ...,Children's Books,Publishing,USD,2015-07-01,5000.0,2015-05-02 09:06:44,5843.00,successful,92,US,5843.00,5843.00,5000.00
15316,107737680,EvoCreo™,Video Games,Games,USD,2014-03-07,6000.0,2014-02-05 16:20:11,2992.25,failed,234,US,2992.25,2992.25,6000.00
168297,1856198403,The Lost Dungeons of Xon: Pathfinder compatibl...,Tabletop Games,Games,USD,2014-07-29,15000.0,2014-06-27 05:20:29,7407.00,failed,194,US,7407.00,7407.00,15000.00
249450,338750243,Help me release my first EP!,Music,Music,USD,2015-07-22,600.0,2015-06-22 17:11:50,75.00,failed,3,US,75.00,75.00,600.00
166768,184826515,The Wounded Healer,Action,Film & Video,USD,2016-11-22,5000.0,2016-09-23 23:08:08,35.00,canceled,1,US,35.00,35.00,5000.00


In [87]:
df.columns.tolist()

['ID',
 'name',
 'category',
 'main_category',
 'currency',
 'deadline',
 'goal',
 'launched',
 'pledged',
 'state',
 'backers',
 'country',
 'usd pledged',
 'usd_pledged_real',
 'usd_goal_real']

In [88]:
df['currency'].value_counts()

USD    295365
GBP     34132
EUR     17405
CAD     14962
AUD      7950
SEK      1788
MXN      1752
NZD      1475
DKK      1129
CHF       768
NOK       722
HKD       618
SGD       555
JPY        40
Name: currency, dtype: int64

In [11]:
df['deadline'].sort_values(ascending=False)

7164      2018-03-03
180250    2018-03-03
226982    2018-03-03
124919    2018-03-02
292034    2018-03-02
             ...    
196244    2009-05-22
28960     2009-05-20
342226    2009-05-16
213711    2009-05-16
138572    2009-05-03
Name: deadline, Length: 378661, dtype: object

In [12]:
df['currency'].value_counts()

USD    295365
GBP     34132
EUR     17405
CAD     14962
AUD      7950
SEK      1788
MXN      1752
NZD      1475
DKK      1129
CHF       768
NOK       722
HKD       618
SGD       555
JPY        40
Name: currency, dtype: int64